In [1]:
import numpy as np                            #Library that has extensive set of math methods and functions.
from scipy.integrate import odeint            #Library that has function for solving differentail equations
from scipy.interpolate import interp1d        #Library containing fuctions of interpolating data
from scipy.optimize import curve_fit          #Library containing fuctions of fitting paramaters to data
from scipy.stats import t as tdist
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import math
from datetime import timedelta
import csv


#Library to plot data
import plotly.graph_objects as go

#använd curvefit för att anpassa parametrar 

mobility_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/SwedenMobilityData.csv')   #imports mobility data from github

temperature_data = pd.read_json('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/smhidata_2020_2021.json')  #imports temperature data from github 

admission_data = pd.read_json('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/Inskrivningsdata.json')                  #imports admission data from github

VT_mobility_data = pd.read_csv('https://raw.githubusercontent.com/Dan-Irl/MVEX-Matematiska-modeller-f-r-smittspridning/main/VT_mobility.csv')


#Jämför RMSE mellan modeller berode på antal datapunkter
#jämför hur ofta prediction hamnar i predictionsintervall mellan modeller berode på antal datapunkter

In [2]:
# A array of time points in weeks for the model to use
prediction_time = 23
t = np.linspace(0, prediction_time, prediction_time)

#Defines how many data point are to be used for model fitting
fittingPoints = 23 # Max 23 weeks
t_fittingdata = np.linspace(0, fittingPoints, fittingPoints)
startdata = 6  #number of intial data points for MAP calculation in weeks




Calcualtions total addmissions active per day from the dailiy admission data

In [3]:
#Calulates the total addmission active for each day by summing the curent day plus the prevoius 4


weeklyAdmissions = []
for week in range(int(math.floor(len(admission_data['Inskrivningar'])/7))):
  week = week*7+1
  
  weeklyAdmissions.append(sum(admission_data['Inskrivningar'][week-8:week-1]))
  
weeklyAdmissions.remove(0)



Created the mobility function for Västtrafik

In [4]:
T = VT_mobility_data['Totalt']
VT_mobilityfunc = interp1d(t,T[:prediction_time],kind='linear',fill_value="extrapolate")

functionfig = go.Figure()

functionfig.add_trace(go.Scatter(x=t, y=VT_mobilityfunc(t),
                    mode='lines',
                    name='Västtrafik mobility change'))
functionfig.update_layout(
                title='Weekly change in mobility %',
                xaxis_title='Weeks',
                yaxis_title='Mobility in %',
                autosize=False,
                width=1000,
                
                font=dict(
                family="Courier New, monospace",
                size=14,
                color="RebeccaPurple"
    )
) 
functionfig.show()

This cell defines the model function and contains most of its dependensies. The SIR model is defined as presendted my Wiess (STORT FEL Gamma är 0.2 vilket är för dagar inte för veckor :())

In [5]:
#Weiss implementering 

# Contact rate, transmisabilty, beta, and mean recovery rate gamma.
#nunmpy quantile för confidensintervall för beta
#försjkut admissions
#Lagen borde inte vara i modelfunction, för den modellerar antalet infected, lagen borde bara vara på predicted admissions

# Total population, N.
N = 700000 #göteborg stad
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0

tau = 1         #transmisability chance
gamma =  1.4    #recovery rate 1/days
lag = 3


# Initial conditions vector
y0 = S0, I0, R0
p = 0.013 #fraction of the present number of infectious individuals

def modelfunction(t,c0,c1): #Enbedded function that takes 
  def betafunc(t):
    kappa =  c0+c1*VT_mobilityfunc(t)
    b = kappa*tau
    beta = b/N
    return beta

  # The SIR model differential equations.
  def deriv(y, t, N):
    S, I, R = y
    dSdt = -betafunc(t-lag) * S * I 
    dIdt = betafunc(t-lag) * S * I - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt
  
  result = odeint(deriv, y0, t, args=(N,))
  S, I, R = result.T
  #Returerar totalt antal inskrivningar varje dag
  return I*p



#Curve fit function that takes the modelfunction with the given parameters to optimize and returns them with optimal value and covariance
popt, pcov = curve_fit(modelfunction, t_fittingdata, weeklyAdmissions[:fittingPoints], p0=[2.7,0.5])

#best VT guess p0=[2.7,0.5])
#Best google guess p0=[3,0.6])

print(popt)
print(pcov)
print(mean_squared_error(weeklyAdmissions,modelfunction(t,*popt), squared = False))

fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=fittingPoints)), 
                    y=weeklyAdmissions,
                    mode='markers',
                    name='Admission data'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=modelfunction(t,*popt),
                    mode='lines',
                    name='Predicted admissions'))
fig.update_layout(
                    title='Time dependant SIR model',
                    xaxis_title='Weeks',
                    yaxis_title='Admissions',
                    autosize=False,
                    width=1000,
                    
                    font=dict(
                    family="Courier New, monospace",
                    size=14,
                    color="RebeccaPurple"
    )
)
fig.show()

[2.93872479 0.02647737]
[[3.30493960e-04 6.00407834e-06]
 [6.00407834e-06 1.96154097e-07]]
22.799546710707208


Plots the predicted admissions and the admission data, for comparison

Plots confidence intervals of model

In [6]:
numBetas = 100 #number of betas used for normal dist for conf. intervalls
numEpsilon = 500

##################  Confidence intervall calculation
confint = np.zeros([prediction_time,2]) #defines confidence intervall arrays


# Generates an array of betas using normal dist.
betas=np.random.multivariate_normal(popt,pcov,numBetas) 
values = np.zeros((prediction_time,numBetas))

for beta in range(len(betas)): 
  values[:,beta] = (modelfunction(t,*betas[beta]))
  
for day in range(len(t)):
  confint[day,0]=np.quantile(values[day,:],0.025)
  confint[day,1]=np.quantile(values[day,:],0.975)
  

##################  Prediction intervall calculation
#få med beta skattning i predictionsintervall, dvs generera beta och fel för varje dag 

std_residual = np.std(modelfunction(t_fittingdata,*popt)-weeklyAdmissions[:fittingPoints], ddof=2) #calcuales standard deviation of residual between model and data
predint = np.zeros([prediction_time,2])

temp = []
pred_matrix = np.zeros((prediction_time,(numBetas*numEpsilon)))
for day in range(prediction_time): #loops trough model values each day a adds the error values generated from t-dist to each prediction 
  for beta in range(len(betas)):
      temp =  np.add(modelfunction(t,*betas[beta])[day],tdist.rvs(prediction_time,0,std_residual,numEpsilon))
      pred_matrix[day,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

for day in range(prediction_time): #calculates the prediction intervals
    predint[day,0] = np.quantile(pred_matrix[day],0.025)
    predint[day,1] = np.quantile(pred_matrix[day],0.975)
    
predint = np.clip(predint,0,None) #removes all negative values from lover prediction interval

fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
fig.add_trace(go.Scatter( #plots the fitting admission data points
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=fittingPoints)), 
                    y=weeklyAdmissions,
                    mode='markers',
                    name='Fitting admission points'))
fig.add_trace(go.Scatter( #plots all the available admission data points
                    x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(weeks=fittingPoints), freq="W", periods=prediction_time-fittingPoints)),
                    y=weeklyAdmissions[fittingPoints:],
                    mode='markers',
                    marker=dict(
                        color='red',
                    ),
                    name='Admission data'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=modelfunction(t,*popt),
                    mode='lines',
                    name='Predicted admissions'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=confint[:,0],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict( width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=confint[:,1],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=predint[:,0],
                    mode='lines',
                    name='Lower prediction interval',
                    line = dict(width=2, dash='dot')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=predint[:,1],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(weeklyAdmissions,modelfunction(t,*popt), squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Västtrafik mobilitets baserad SIR modell',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01","2020-08-02"],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()

Cell calcualtes the cumulative admissions and confidens/prediktion intervalls

In [7]:
numBetas = 100 #number of betas used for normal dist for conf. intervalls
numEpsilon = 500

##################  Confidence intervall calculation
confint = np.zeros([prediction_time,2]) #defines confidence intervall arrays


# Generates an array of betas using normal dist.
betas=np.random.multivariate_normal(popt,pcov,numBetas) 
values = np.zeros((prediction_time,numBetas))

for beta in range(len(betas)): 
  values[:,beta] = np.cumsum(modelfunction(t,*betas[beta]))
  
for day in range(len(t)):
  confint[day,0]=np.quantile(values[day,:],0.025)
  confint[day,1]=np.quantile(values[day,:],0.975)
  

##################  Prediction intervall calculation
#få med beta skattning i predictionsintervall, dvs generera beta och fel för varje dag 

std_residual = np.std(modelfunction(t_fittingdata,*popt)-weeklyAdmissions[:fittingPoints], ddof=2) #calcuales standard deviation of residual between model and data
predint = np.zeros([prediction_time,2])

temp = []
pred_matrix = np.zeros((prediction_time,(numBetas*numEpsilon)))
for day in range(prediction_time): #loops trough model values each day a adds the error values generated from t-dist to each prediction 
  for beta in range(len(betas)):
      temp =  np.add(np.cumsum(modelfunction(t,*betas[beta]))[day],tdist.rvs(prediction_time,0,std_residual,numEpsilon))
      pred_matrix[day,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

for day in range(prediction_time): #calculates the prediction intervals
    predint[day,0] = np.quantile(pred_matrix[day],0.025)
    predint[day,1] = np.quantile(pred_matrix[day],0.975)
    
predint = np.clip(predint,0,None) #removes all negative values from lover prediction interval

fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
fig.add_trace(go.Scatter( #plots the fitting admission data points
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=fittingPoints)), 
                    y=np.cumsum(weeklyAdmissions),
                    mode='markers',
                    name='Inskrivningsdata'))
fig.add_trace(go.Scatter( #plots all the available admission data points
                    x=pd.Series(pd.date_range(pd.to_datetime('2020-03-01')+timedelta(weeks=fittingPoints), freq="W", periods=prediction_time-fittingPoints)),
                    y=np.cumsum(weeklyAdmissions)[fittingPoints:],
                    mode='markers',
                    marker=dict(
                        color='firebrick',
                    ),
                    name='Admission data'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=np.cumsum(modelfunction(t,*popt)),
                    mode='lines',
                    name='predikterade kumulativa inskrivningar'))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=confint[:,0],
                    mode='lines',
                    name='Undre konfidensintervall',
                    line = dict( width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=confint[:,1],
                    mode='lines',
                    name='Övre konfidensintervall',
                    line = dict(width=2, dash='dash')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=predint[:,0],
                    mode='lines',
                    name='Undre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
fig.add_trace(go.Scatter(
                    x=pd.Series(pd.date_range("2020-03-01", freq="W", periods=prediction_time)), 
                    y=predint[:,1],
                    mode='lines',
                    name='Övre prediktionsintervall',
                    line = dict(width=2, dash='dot')))
fig.add_annotation(x=pd.to_datetime("2020-03-01"),y=0, 
                   text=f"RMSE: {math.floor(mean_squared_error(weeklyAdmissions,modelfunction(t,*popt), squared = False))}",
                   showarrow=False,
                   bordercolor="black",
                   xshift=48,
                   yshift=270
                  )
fig.update_layout(
                    title='Västtrafik mobilitets baserad SIR modell',
                    xaxis_title='Datum',
                    yaxis_title='Inskrivningar',
                    autosize=False,
                    width=1000,
                    xaxis_range=["2020-03-01",pd.to_datetime('2020-03-01')+timedelta(weeks=prediction_time-1)],
                    
                    font=dict(
                      family="Courier New, monospace",
                      size=14,
                      color="RebeccaPurple"
                    )
)
fig.show()

Calculate MAPE of the model as the amout of fitting data for curve fit increases

In [8]:
header = ['Idex', 'Fitting points', 'MAPE'] #header for csv file
RMSEovertime = []   #init RMSE array


#writes a csv file of Index, number of fitting points and MAP
with open('SIR_MAPE_weekly.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    
    #loops over all data point for fitting and curve fits the model and calculates MAPE
    for week in range(len(weeklyAdmissions)):
        #breaks loop when reaching final value
        if week+startdata+3 > len(weeklyAdmissions):
            break
        
        #inits temp array for writing to csv file and time array for model
        temp_RMSEovertime = []
        t_MAPE1 = np.linspace(0,week+startdata,week+startdata)
        t_MAPE2 = np.linspace(0,week+startdata+3,week+startdata+3)
        
        #curve fits model to fitting data
        MAPE_popt = curve_fit(modelfunction, t_MAPE1, weeklyAdmissions[:week+startdata], p0=[2.7,0.5])
        
        
        #creates temp list of Index, number of fitting points and MAP
        temp_RMSEovertime.append(week)
        temp_RMSEovertime.append(week+startdata)
        temp_RMSEovertime.append(mean_absolute_percentage_error(np.cumsum(weeklyAdmissions[:week+startdata+3])[-3:],np.cumsum(modelfunction(t_MAPE2,*MAPE_popt))[-3:]))
        
        #Appends matrix of all MAP vallues
        RMSEovertime.append(temp_RMSEovertime)
        
        # write the data
        writer.writerow(temp_RMSEovertime)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

Cell that calcualtes the IS_alpha for the model

In [ ]:
numBetas = 100 #number of betas used for normal dist for conf. intervalls
numEpsilon = 500
alpha = 0.05
ISalpha_score = []

header = [ 'Fitting points', 'ISalpha'] #header for csv file


with open('SIR_ISalpha_weekly.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    for week in range(len(weeklyAdmissions)):
            #breaks loop when reaching final value
            if week+startdata+3 > len(weeklyAdmissions):
                break
            t_ISalpha1 = np.linspace(0,week+startdata,week+startdata)       #used for fitting
            t_ISalpha2 = np.linspace(0,week+startdata+3,week+startdata+3)     #used for residual calculation
            
            #curve fits model to fitting data
            ISalpha_popt, ISalpha_pcov = curve_fit(modelfunction, t_ISalpha1, weeklyAdmissions[:week+startdata], p0=[2.7,0.5])
            
            #calcuales standard deviation of residual between model and data using the fitting data + 3 weeks forward
            std_residual_ISalpha = np.std(modelfunction(t_ISalpha1,*ISalpha_popt)-weeklyAdmissions[:week+startdata], ddof=2)
            # Generates an array of betas using normal dist.
            ISalpha_betas=np.random.multivariate_normal(ISalpha_popt,ISalpha_pcov,numBetas)  
            
            #Generates the prediction matrix for the prediction interval
            predint_ISalpha = np.zeros([3,3])
            
            temp = []
            pred_matrix = np.zeros((3,(numBetas*numEpsilon)))
            for weekInInterval in range(3): #loops trough model values each week a adds the error values generated from t-dist to each prediction 
                for beta in range(len(ISalpha_betas)):
                    temp =  np.add(np.cumsum(modelfunction(t_ISalpha2,*ISalpha_betas[beta]))[week+startdata+weekInInterval],tdist.rvs(weekInInterval+startdata+3,0,std_residual_ISalpha,numEpsilon))
                    pred_matrix[weekInInterval,(beta*numEpsilon):(beta*numEpsilon+numEpsilon)] = temp

            for weekInInterval in range(3): #calculates the prediction intervals
                predint_ISalpha[weekInInterval,0] = np.quantile(pred_matrix[weekInInterval],0.025)
                predint_ISalpha[weekInInterval,1] = np.quantile(pred_matrix[weekInInterval],0.975)
                predint_ISalpha[:,2] = np.cumsum(modelfunction(t_ISalpha2,*ISalpha_betas[beta]))[-3:]
                
            predint_ISalpha = np.clip(predint_ISalpha,0,None) #removes all negative values from lover prediction interval
            
            #predint_ISalpha now contains lower, upper and the model output (columms) for the 3 weeks (rows)
            
            temp_score = []

            #calculates the score for each week of the 3 week interval and then avrages them
            for weekInInterval,value in enumerate(predint_ISalpha):

                u = predint_ISalpha[weekInInterval,0]
                l = predint_ISalpha[weekInInterval,1]
                y = predint_ISalpha[weekInInterval,2]

                coeff1 = 1 if y < l else 0
                coeff2 = 1 if y > u else 0

                
                temp_score.append((u-l)+2/alpha*(l-y)*coeff1+2/alpha*(y-u)*coeff2)

            temp_ISalpha_score = [week+startdata-3, np.average(temp_score)]
            ISalpha_score.append(temp_ISalpha_score)
            
            # write the data
            writer.writerow(temp_ISalpha_score)

Calcualates the final cumulative value of of the model as fitting points increase

In [18]:
header = ['Idex', 'Fitting points', 'Total admissions'] #header for csv file
Totaladmissionsovertime = []   #init RMSE array


#writes a csv file of Index, number of fitting points and MAP
with open('Totaladmissions_weekly.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    
    #loops over all data point for fitting and curve fits the model and calculates MAPE
    for week in range(len(weeklyAdmissions)):
        #breaks loop when reaching final value
        if week+startdata+3 > len(weeklyAdmissions):
            break
        
        #inits temp array for writing to csv file and time array for model
        temp_totaladmissions = []
        t_totaladmissions = np.linspace(0,week+startdata,week+startdata)
        
        #curve fits model to fitting data
        TotalAdmissions_popt, TotalAdmissions_covt = curve_fit(modelfunction, t_totaladmissions, weeklyAdmissions[:week+startdata], p0=[2.7,0.5])
        
        print(TotalAdmissions_popt)
        
        #creates temp list of Index, number of fitting points and MAP
        temp_totaladmissions = [week+startdata-3,np.cumsum(modelfunction(t,*TotalAdmissions_popt))[-1]]
        
        
        #Appends matrix of all MAP vallues
        Totaladmissionsovertime.append(temp_RMSEovertime)
        
        # write the data
        writer.writerow(temp_totaladmissions)

(array([2.77838401, 0.02649503]), array([[2.08451127e-04, 6.24421989e-05],
       [6.24421989e-05, 2.69026275e-05]]))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [16]:
pd.to_datetime('2020-03-01')+timedelta(weeks=21)
np.cumsum(modelfunction(t,*[2.7,0.5]))[-1]

3984.182991716148

Cell that seatches for a global minimum of the paramaters given using RMSE

In [ ]:
#VT global param search
paramC0 = np.linspace(0,4,50)  #x-values
paramC1 = np.linspace(0,1,50)  #y-values

# #Google global param search
# params.append(np.linspace(0,4,50)) 
# params.append(np.linspace(0,1,50))

result = np.zeros((len(paramC0),len(paramC1)))
for param1 in range(len(paramC0)):
    for param0 in range(len(paramC1)):
        result[param0,param1] = math.log(mean_squared_error(weeklyAdmissions,modelfunction(t,paramC0[param0],paramC1[param1]), squared = False))



fig = go.Figure() #Creates a plot using graph objects and then adds each graph to figure
# fig.add_trace(go.Scatter(x=t, y=weeklyAdmissions,
#                     mode='lines',
#                     name='Admission data'))
# fig.add_trace(go.Scatter(x=t, y=modelfunction(t,*best[0]),
#                     mode='lines',
#                     name='Predicted admissions'))

fig.add_contour(
                    z=result,
                    x=paramC0,
                    y=paramC1,
                    colorbar=dict(
                        titleside='right',
                        title='Log RMSE', # title here
                    )
)  
  

fig.update_layout(
                    title='Global parameter search Google mobility data',
                    xaxis_title='c0 value',
                    yaxis_title='c1 value',
                    autosize=False,
                    width=800,
                    height=800,
                    font=dict(
                        family="Courier New, monospace",
                        size=18,
                        color="RebeccaPurple"
    )
)
fig.show()

#Countour plot plotta log(RMSE) för Z axeln
